# Collaborative based reccomenders
---
_Project by Qijun Jin, Johhny Nuñez and Marcos Plaza._

### The goal

This first practice of the course, is based on using the data from this [kaggle](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/overview) competition to obtain a **product recommendation** based on customer transaction data, as well as other metadata.

To achieve this goal, we have relied on **collaborative methods**. The hypothesis is: _**"Similar users tend to like similar items"**_. More precisely, this is the definition of Neighborhood-based methods (that were among the earliest algorithms developed for collaborative filtering. The previous sentece is valid for a user-based system). So, for this problem as we have a lot of users, we have focused our efforts on proposing a **item-based model**.

First of all let's check the data to gain some insights:

In [1]:
import numpy as np
import pandas as pd 
import os
from tqdm.notebook import tqdm

# load data
original_df_customers=pd.read_csv('./data/customers.csv')
original_df_items=pd.read_csv('./data/articles.csv')
original_df_customers_items=pd.read_csv('./data/transactions_train.csv')

### Information of the customers

In [2]:
original_df_customers.head(10)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


### Information of the items

In [3]:
original_df_items.head(10)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,110065011,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
7,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
9,111593001,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


### Information of the transactions: Which customers have purchased certain items?

In [4]:
original_df_customers_items.head(10)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
5,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2
6,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2
7,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,0.030492,1
8,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,0.053373,1
9,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,0.045746,2


This is the data set on which we will focus our implementation. As we can see, there are different metadata that mainly tell us which customer bought a certain item. On the other hand, the number of transactions is huge, so it will not be feasible to use the whole dataset at a computational level. 

Therefore, we must force a good enough limit to give a good recommendation. We thought it was a good idea to **take into account the most recent transactions**, since products such as clothing tend to renew and change over time and seasons. We will discard all products before to the following date: ``2020-08-31``.


In [5]:
d = original_df_customers_items.copy()

original_df_customers_items = d[d['t_dat'] > '2020-08-31']

counts_df = original_df_customers_items.groupby(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']).size()
counts_df = counts_df.to_frame()
counts_df.reset_index(inplace=True)

small_counts = counts_df.rename(columns={0: 'count'})

In [6]:
small_counts

,t_dat,customer_id,article_id,price,sales_channel_id,count
0,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,777148006,0.013542,1,1
1,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,835801001,0.018627,1,1
2,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,865929003,0.016932,1,1
3,2020-09-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,923134005,0.012695,1,1
4,2020-09-01,0005ed68483efa39644c45185550a82dd09acb07622acb...,863646004,0.033881,1,1
...,...,...,...,...,...,...
727329,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,1
727330,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,1
727331,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,1
727332,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,1


### Implementation

The first thing we have to do is to build a matrix that will serve, in some way, as an indication of each person's preferences. For this purpose, we will construct a matrix $m$×$n$, where $m$ is the number of customers and $n$ the number of items or articles. In every entry $(i,j)$ of this matrix we will have the number of times that customer have bought this article.

In [7]:
from scipy.sparse import csr_matrix, dok_matrix
from pandas.api.types import CategoricalDtype

def to_dense(array):
    """
    Accepta una csr_matrix, dok_matrix o matrix i la converteix en una 
    np.array normal, densa.
    
    :param array: Array a convertir
    :return: np.array densa, sense cap dimensió de tamany 1
    """
    try:
        array = array.todense()
    except:
        pass
    
    return np.array(array).squeeze()
    
def build_counts_table(df):
    """
    Retorna una csr_matrix on les columnes són els `items`, les files `customer_id` i els valors
    el nombre de vegades que un usuari ha escoltat un `item`
    
    :param df: DataFrame original després de creuar-lo
    :return: Una tupla constistent de:
        * La csr_matrix descrita
        * Els indexos corresponents a cada fila (el customerID de la fila `i` corresponent a l'element `i` d'aquesta array)
        * Les columnes corresponents a cada columna (el article_id de la columna `j` correspon a l'element `j` d'aquesta array)
    """
    # Ids, sense repeticions i ordenats
    customer_ids = CategoricalDtype(sorted(df.customer_id.unique()), ordered=True)
    item_ids = CategoricalDtype(sorted(df.article_id.unique()), ordered=True)

    # Conversió a csr
    row = df.customer_id.astype(customer_ids).cat.codes
    col = df.article_id.astype(item_ids).cat.codes
    sparse_matrix = csr_matrix((df["count"], (row, col)), \
                           shape=(customer_ids.categories.size, item_ids.categories.size))

    return sparse_matrix, customer_ids, item_ids

### Top 10 customers & Top 10 items

In [8]:
def top_active_customers(counts, indexes, columns, n):
    """
    Exemple: Retorna els ids dels n usuaris que més reproduccions han acumulat
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param n: Quanitat d'usuaris
    :return: Llista, tupla o pd.Series de customerID dels n usuaris
    """
    # Operate with the sparse matrix, convert to dense the result (as it has much fewer entries)
    sums = to_dense(counts.sum(axis=1))
    # Get indices
    indices = sums.argsort()
    return indexes.categories[indices[-n:]]

def top_bought_articles(counts, indexes, columns, n):
    """
    Exemple: Retorna els ids dels n itemes més escoltats
    
    :param counts, indexes, columns: Tupla retornada per `build_counts_table`
    :param n: Quanitat d'itemes
    :return: Llista, tupla o pd.Series de itemID dels n itemes
    """
    # Operate with the sparse matrix, convert to dense the result (as it has much fewer entries)
    sums = to_dense(counts.sum(axis=0))
    # Get indices
    indices = sums.argsort()
    return columns.categories[indices[-n:]]

In [9]:
counts, indexes, columns = build_counts_table(small_counts)

This is the top 10 customers who have made the most purchases. On the other hand we have the 10 most purchased products (within the restrictions that have been applied).

In [10]:
top_customers = top_active_customers(counts, indexes, columns, 10)
top_customers

Index(['29faf622c66c95935e11709b28bc3252de560ec9fcb76e53e82bd877f3a7f447',
       '4de6988c4971cfc0c6910a85a54fa8d6fb11be64b87b98c46b73821d7db07ce7',
       '54e8ebd39543b5a4d69c3e7d79977558d2a606e6540ba0a50e07001cfff202c2',
       '75ebdc56559b1f2739ce5832bd85a921ba827c72383135bdcc08a616d320e948',
       'e4ea6ece6706e9c119a3640e09e842f5ae7d62a6b546d4f2448e08def4bd7283',
       'd1c5629d6d7e39067cef9d1092e44016ea94889fa36624fad0123c39de72edb1',
       '1be8d175dddc1efc0abd9d10c95d1bc5ecef4d649ed8b0a7b4c878f4e2a425ac',
       '3860b5e65d48bb509d89e6b21ec0458e13c75eab2e95f53ea2269fafc01a5567',
       '1426b606af826fa9568b6fe75629d10bb6ff7ea40f93908c43113d991e512bd5',
       '65cb62c794232651e2ac711faa11c2b4e3d41d5f3b59b50bee3ffde1d5776644'],
      dtype='object')

In [11]:
top_items = top_bought_articles(counts, indexes, columns, 10)
top_items

Int64Index([706016001, 751471043, 915529003, 448509014, 915526001, 918292001,
            924243001, 918522001, 909370001, 751471001],
           dtype='int64')

At the same time, we have set another limit to further reduce the dimensionality of our data. We will **keep those top items and customers**.

In [12]:
counts, indexes, columns = build_counts_table(small_counts)
print(counts.shape)
top_customers = top_active_customers(counts, indexes, columns, 5000)
top_items = top_bought_articles(counts, indexes, columns, 5000)


s = small_counts.copy()
print("Total: ", len(s))
s = s[s.article_id.isin(top_items)]
print("Filter top articles: ", len(s))
s = s[s.customer_id.isin(top_customers)]
print("Filter top customers: ", len(s))
# s = s[:10000]
print("Total: ", len(s))
s

(189510, 26252)
Total:  727334
Filter top articles:  598564
Filter top customers:  74509
Total:  74509


,t_dat,customer_id,article_id,price,sales_channel_id,count
120,2020-09-01,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,237347059,0.033881,2,2
121,2020-09-01,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,237347063,0.033881,2,2
122,2020-09-01,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,441386001,0.042356,2,2
228,2020-09-01,01661bad253b21353092eaf49836913843e1fc0554a674...,889460003,0.025407,2,2
250,2020-09-01,01959be607170cc2f092ee8fd13eda251b13cde70ef38d...,506098006,0.030492,2,2
...,...,...,...,...,...,...
727326,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,896169005,0.050831,2,1
727327,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,902288001,0.022017,2,1
727328,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,910949002,0.050831,2,1
727329,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,1


In [13]:
s = s.drop(s.columns[[0, 3, 4]], axis=1)
s

,customer_id,article_id,count
120,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,237347059,2
121,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,237347063,2
122,009a85913aa6f503ed0d2b5ac02ab919d6565bbbaa934a...,441386001,2
228,01661bad253b21353092eaf49836913843e1fc0554a674...,889460003,2
250,01959be607170cc2f092ee8fd13eda251b13cde70ef38d...,506098006,2
...,...,...,...
727326,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,896169005,1
727327,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,902288001,1
727328,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac...,910949002,1
727329,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,1


In [14]:
counts, indexes, columns = build_counts_table(s) # build the counts matrix

### Compute similarities

To make collaborative recommendation there are two options, to make a user-based recommender or an item-based recommender. As we said in the introduction, we wanted to give an approach to an **item-based system**:
+ On an item-based approach, we will consider the matrix N×M (items×users), in order to recommend, you will have to base your recommendation on the similarities between the items.

To compute the product-based similarity matrix, we only need to transpose the ``counts`` matrix above (``counts.T``). This will give us a triangular matrix of N x M based on items.

In [15]:
from sklearn.metrics.pairwise import pairwise_distances

def similarity_matrix(similarity_function, counts):
    if similarity_function is None:
        x = to_dense(counts)
        y = to_dense(counts.T)
        
        matrix_prod = np.dot(x,y)
        diagonal = np.diag(matrix_prod)
        inversa = 1 / diagonal
        inversa[np.isinf(inversa)] = 0
        inv_mag = np.sqrt(inversa)
        
        cosine = matrix_prod * inv_mag
        cosine = cosine.T * inv_mag
        np.fill_diagonal(cosine, 0)
        
        return cosine
    
    else: 
        print("matrix")
        x = pd.DataFrame.sparse.from_spmatrix(counts)
        matrix = pairwise_distances(X = x, metric = similarity_function, n_jobs = -1)
        del x
        matrix = csr_matrix(matrix)
                
        return matrix

In [16]:
import pickle

try:
    with open('similarities.pkl', 'rb') as fp:
        similarities = pickle.load(fp)
except:
    similarities = similarity_matrix(similarity_function="correlation", counts=counts.T)
        
    with open('similarities.pkl', 'wb') as fp:
        pickle.dump(similarities, fp, pickle.HIGHEST_PROTOCOL)

### Make a prediction

To make a collaborative recommendation, we need a function that gives us a value of how good the recommendation would be. In our case, as it is item-based, **the score** for a user $u$ i item $i$ is:
$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{j\neq i,r_{u,j}>0} sim(i, j)\cdot r_{u,j}}{\sum_{j\neq i,r_{u,j}>0} sim(i, j)}$$

In [17]:
def score(counts, indexes, columns, similarities, customer, item):
    customer_ = indexes.categories.get_loc(customer)
    item_ = columns.categories.get_loc(item)
    
    rui = to_dense(counts[:,customer_])
    
    sim = np.triu(to_dense(similarities[item_]), k=1)
    sim = sim[item_]
    
    numerador = np.sum(rui* sim)
    denominador = np.sum(sim[rui > 0])
    
    if denominador != 0 and numerador != 0:
        return numerador/denominador
    else:
        return 0

In [18]:
print(score(counts.T, indexes, columns, similarities, '01959be607170cc2f092ee8fd13eda251b13cde70ef38dd37e37dcdadfde3b9e',781758057)) #111,123

0.9999999999999999


In [19]:
def score_mean(counts, indexes, columns, similarities, customer, item):
    customer_ = indexes.categories.get_loc(customer)
    item_ = columns.categories.get_loc(item)
    ru = np.mean(counts[:,item_])
    ri = to_dense(counts.mean(axis=1))
    rpi = to_dense(counts[:,customer_])
    sim = np.triu(to_dense(similarities[item_]), k=1)
    sim = sim[item_]
    diferencia = np.subtract(rpi,ri)[rpi>0]
    numerador = np.sum(diferencia *sim[rpi > 0])
    denominador = np.sum(sim[rpi > 0])
    if numerador !=0 and denominador !=0:
        return ru + (numerador/denominador)
    else:
        return ru

In [20]:
print(score_mean(counts.T, indexes, columns, similarities, '01959be607170cc2f092ee8fd13eda251b13cde70ef38dd37e37dcdadfde3b9e',781758057)) 


0.9890914008781669


In [21]:
from queue import PriorityQueue
from tqdm.notebook import trange, tqdm
import heapq as hq
import operator

def recommend_n_items(counts, indexes, columns, similarities, customer, N):
    minheap = []
    customer_ = indexes.categories.get_loc(customer)
    for item in tqdm(columns.categories.tolist()):
        item_ = columns.categories.get_loc(item)
        if counts[customer_,item_] == 0:
            valor = score(counts, indexes, columns, similarities, customer, item)
            hq.heappush(minheap, (-valor, item))

    res = []

    for i in range(N):
        res.append(hq.heappop(minheap)[1])

    return res

In [22]:
print(recommend_n_items(counts.T, indexes, columns, similarities, '01959be607170cc2f092ee8fd13eda251b13cde70ef38dd37e37dcdadfde3b9e',12)) 

  0%|          | 0/4903 [00:00<?, ?it/s]

[764147001, 762143010, 762143002, 759871030, 762846029, 766402023, 762846006, 762846026, 762143009, 762846001, 761406004, 762846031]


In [23]:
from queue import PriorityQueue
import heapq as hq
import operator

def recommend_n_items_mean(counts, indexes, columns, similarities, customer, N):
    minheap = []
    customer_ = indexes.categories.get_loc(customer)
    for item in tqdm(columns.categories.tolist()):
        item_ = columns.categories.get_loc(item)
        if counts[customer_,item_] == 0:
            valor = score_mean(counts, indexes, columns, similarities, customer, item)
            hq.heappush(minheap, (-valor, item))

    res = []

    for i in range(N):
        res.append(hq.heappop(minheap)[1])

    return res

In [24]:
print(recommend_n_items_mean(counts.T, indexes, columns, similarities, '01959be607170cc2f092ee8fd13eda251b13cde70ef38dd37e37dcdadfde3b9e',12))

  0%|          | 0/4903 [00:00<?, ?it/s]

[762143009, 764147001, 762796004, 763988011, 764073002, 762846031, 766402023, 763275002, 762143010, 760084013, 762600013, 760084015]


In [25]:
def Convert(string):
    li = list(string.split(" "))
    return li

### Generate the ``submission.csv`` file

In [26]:
kaggle_df_customer_articles = pd.read_csv('./data/sample_submission.csv')

In [27]:
kaggle_df_customer_articles.customer_id.unique()

array(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       '0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [ ]:
try:
    from tqdm.notebook import trange, tqdm
except:
    def tqdm(a, _, *__): return a
    
# Fetch kaggle public data and merge
kaggle_df_customer_articles = pd.read_csv('./data/sample_submission.csv')
    
# Obtain counts
pred_counts, pred_indexes, pred_columns = build_counts_table(s)
    
# Similarity
try:
    with open('pred_similarities.pkl', 'rb') as fp:
        pred_similarities = pickle.load(fp)
except:
    pred_similarities = similarity_matrix(similarity_function="correlation", counts=pred_counts.T)
        
    with open('pred_similarities.pkl', 'wb') as fp:
        pickle.dump(pred_similarities, fp, pickle.HIGHEST_PROTOCOL)
    
results = pd.DataFrame(columns=['customer_id', 'prediction'])
top_n_items = top_bought_articles(pred_counts, pred_indexes, pred_columns, 12)
top_n_items = ''.join(['0'+str(i)+' ' for i in top_n_items])[:-1]

for idx, customer in enumerate(tqdm(kaggle_df_customer_articles.customer_id.unique())):
    try:
        article_ids = recommend_n_items_mean(pred_counts.T, pred_indexes, pred_columns, pred_similarities, customer,12)
        article_ids = Convert(article_ids)
        article_ids = ''.join(['0'+str(i)+' ' for i in article_ids])[:-1]
    except:
        article_ids = top_n_items
    results.loc[idx] = (customer, ''.join(article_ids))
        
results.to_csv('submission.csv', index=False)

  0%|          | 0/1371980 [00:00<?, ?it/s]

  0%|          | 0/4903 [00:00<?, ?it/s]